In [761]:
from pyomo.environ import *

m = ConcreteModel()

n_steps = 50
n_nodes = 4

m.i_steps = Set(initialize=[x for x in range(n_steps)], 
                ordered=True)
m.i_nodes = Set(initialize=[x for x in range(n_nodes+1)], 
                ordered=True)
m.i_nodes_free = Set(initialize=[x for x in range(n_nodes)], 
                ordered=True)
m.i_nodes_free2 = Set(initialize=m.i_nodes_free, 
                ordered=True)

p_nodes_ini = {}
p_nodes_ini[0] = 0.0
p_nodes_ini[1] = 0.25
p_nodes_ini[2] = 0.5
p_nodes_ini[3] = 0.75
p_nodes_ini[4] = 1.0
m.p_nodes = Param(m.i_nodes, 
                  initialize=p_nodes_ini)
m.v_x  = Var(m.i_steps, m.i_nodes_free)
m.v_z  = Var(m.i_steps, m.i_nodes_free)
m.v_u = Var(m.i_steps)


In [762]:
# Functions to define lagrange polynomial
def l_coeff(tstep, k):
    """
    Returns L_k(tstep).
        
    """
    
    v = 1
    for j in [j for j in range(len(m.i_nodes_free)) if j != k]:
        v = v * (m.p_nodes[tstep] - m.p_nodes[j])/(m.p_nodes[k] - m.p_nodes[j])
    return v

# def l(x, tstep):
#     """
#     Returns the lagrange polynomial representation of a variable x
#     """
#     return sum(x[m.i_steps, i] * l_coeff(tstep, i) for i in m.i_nodes if i !=m.i_nodes.last())


# Functions to define derivative of lagrange polynomial

def dot_l_coeff(tstep, j):
    """
    Returns the derivative of L_j(tstep), wher j is a node
    """
    v = 0
    for i in [i for i in range(len(m.i_nodes_free)) if i != j]:    
        c = 1
        for k in [k for k in range(len(m.i_nodes_free)) if (k != i and k != j)]:
            c = c * (m.p_nodes[tstep] - m.p_nodes[k])/(m.p_nodes[j] - m.p_nodes[k])
        c = c /(m.p_nodes[j] - m.p_nodes[i])
        v = v + c
    return v
        
# def dot_l(x, tstep, i_steps):
#     """
#     Returns the derivative of lagrange polynomial representation of variable x
#     """
#     return sum(x[i_steps, i] * dot_l_coeff(tstep, i) for i in m.i_nodes if i !=m.i_nodes.last())


In [763]:
def rule_dot_z(m, i_steps, i_nodes_free):
    if i_nodes_free == m.i_nodes_free.first():
        return Constraint.Skip
    else:
        return sum(
            m.v_z[i_steps, i] * dot_l_coeff(i_nodes_free, i) 
            for i in m.i_nodes_free2
        ) == 0.5 * m.v_u[i_steps]**2
           
m.c_dot_z = Constraint(m.i_steps, m.i_nodes_free, rule=rule_dot_z)


def rule_z_continuity(m, i_steps):
    if i_steps == m.i_steps.last():
        return Constraint.Skip
    else:
        return sum(
            m.v_z[i_steps, i] * l_coeff(m.i_nodes.last(), i) 
            for i in m.i_nodes_free2
        ) == sum(
            m.v_z[i_steps+1, i] * l_coeff(m.i_nodes.first(), i) 
            for i in m.i_nodes_free2 
        )
m.c_z_continuity = Constraint(m.i_steps, rule=rule_z_continuity)


def rule_z_start(m):
    return m.v_z[m.i_steps.first(), m.i_nodes_free.first()] == 0.0
m.c_z_start = Constraint(rule=rule_z_start)


def rule_dot_x(m, i_steps, i_nodes_free):
    if i_nodes_free == m.i_nodes_free.first():
        return Constraint.Skip
    else:
        return n_steps * sum(
            m.v_x[i_steps, i] * dot_l_coeff(i_nodes_free, i) 
            for i in m.i_nodes_free2
        ) == - sum(
            m.v_x[i_steps, i] * l_coeff(i_nodes_free, i) 
            for i in m.i_nodes_free2
        ) + m.v_u[i_steps]
m.c_dot_x = Constraint(m.i_steps, m.i_nodes_free, rule=rule_dot_x)


def rule_x_continuity(m, i_steps):
    if i_steps == m.i_steps.last():
        return Constraint.Skip
    else:
        return sum(
            m.v_x[i_steps, i] * l_coeff(m.i_nodes.last(), i) 
            for i in m.i_nodes_free2
        ) == sum(
            m.v_x[i_steps+1, i] * l_coeff(m.i_nodes.first(), i) 
            for i in m.i_nodes_free2 
        )
m.c_x_continuity = Constraint(m.i_steps, rule=rule_x_continuity)


def rule_x_start(m):
    return m.v_x[m.i_steps.first(), m.i_nodes_free.first()] == 1.0
m.c_x_start = Constraint(rule=rule_x_start)


def rule_x_end(m):
    return sum(
        m.v_x[m.i_steps.last(), i] * l_coeff(m.i_nodes.last(), i) 
        for i in m.i_nodes_free2
    ) == 0.0
m.c_x_end = Constraint(rule=rule_x_end)


# def rule_u_start(m, i):
#     return m.v_u[i] <= 1.0
# m.c_u_start = Constraint(m.i_steps, rule=rule_u_start)

# def rule_u_end(m, i):
#     return m.v_u[i] >= -1.0
# m.c_u_end = Constraint(m.i_steps, rule=rule_u_end)

# def rule_u_ini(m):
#     return m.v_u[m.i_steps.first()] == 2.0
# m.c_u_ini = Constraint(rule=rule_u_ini)

# def rule_x_start(m):
#     return m.v_x[m.i_steps.first(), m.i_nodes_free.first()] == 1.0
# m.c_x_start = Constraint(rule=rule_x_start)

def rule_obj(m):
    return sum(
        m.v_z[m.i_steps.last(), i] * l_coeff(m.i_nodes.last(), i) 
        for i in m.i_nodes_free2
    )
m.obj = Objective(rule=rule_obj)



In [765]:
opt = SolverFactory('ipopt')
run = opt.solve(m, tee=True) 

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1996
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       50

Total number of variables............................:      450
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [766]:
for i in m.i_steps:
    for j in m.i_nodes_free:
        print(value(m.v_x[i,j]))
        
print('u')
for i in m.i_steps:
    print(value(m.v_u[i]))

1.0
0.9934354396247864
0.9869036197928696
0.9804043776159745
0.9739375502058268
0.9674711185180004
0.9610369379596571
0.9546348480774152
0.9482646884178934
0.9418937987585593
0.9355546837157744
0.929247185206869
0.9229711451491733
0.9166932490775206
0.9104466638197537
0.9042312336006793
0.898046802645104
0.891859388918996
0.8857028347288539
0.8795769865446481
0.8734816908363489
0.8673822844077954
0.8613132985815127
0.8552745820112213
0.8492659833506414
0.8432521443757419
0.8372682992380663
0.831314298714544
0.8253899935821043
0.8194593164450837
0.8135582183781737
0.807686552221822
0.801844170816476
0.7959942831674007
0.7901735716533459
0.7843818911194114
0.7786190964106955
0.7728476582165056
0.7671050048930892
0.7613909932321337
0.7557054800253261
0.7500101826337902
0.7443432903631121
0.7387046618942829
0.7330941559082927
0.7274727211245144
0.721879323074106
0.7163138222708317
0.7107760792284546
0.7052262584035661
0.6997041171396268
0.6942095177273778
0.68874232245756
0.683261895589231